# Week 1 - Measuring Meaning & Sampling
This week, we begin by "begging, borrowing and stealing" text from several
contexts of human communication (e.g., PDFs, HTML, Word) and preparing it for
machines to "read" and analyze so that we can begin to build our sample. This notebook outlines scraping text from the web, PDF and Word documents. Then we detail "spidering" or walking
through hyperlinks to build samples of online content, and using APIs,
Application Programming Interfaces, provided by webservices to access their
content. Along the way, we will use regular expressions, outlined in the
reading, to remove unwanted formatting and ornamentation. Next, we discuss
various text encodings, filtering and data structures in which text can be
placed for analysis. Finally, we ask you to begin building a corpus for preliminary analysis and articulate what your sample represents in context of your final project. 

We made a python package just for this course: lucem_illud. If you haven't installed this package, you should run the following code first. You don't need to install the package later; all you need to do is just to import the package with: import lucem_illud. For your final projects, you may find it useful to [read the lucem_illud source code](https://github.com/UChicago-Computational-Content-Analysis/lucem_illud/tree/main/lucem_illud) and modify your code for your own interests.

In [ ]:
!pip install git+https://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git
#installing lucem_illud package
#lucem_illud is a Latin phrase meaning "that light", the insight we can discover in text data!
#If you get an error like "Access is denied", try running the `pip` command on the command line as an administrator.

If you're not familiar with jupyter notebook, you may wonder what the exclamation mark(!) at the beginning of the command does (or even what pip means). The exclamation mark enables us to execute Terminal commands in the notebook cells (e.g., run `!ls` to display files in the current folder).

There is also a special download required by the `lucem_illud` module in the module `spacy`. You will see this 'en' module later, but you should probably run the following 2 lines of code so you can import `lucem_illud`.

In [6]:
import spacy
!python -m spacy download en

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the full
pipeline package name 'en_core_web_sm' instead.
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.1/12.8 MB 2.8 MB/s eta 0:00:05
     -- ------------------------------------- 0.7/12.8 MB 7.4 MB/s eta 0:00:02
     --- ------------------------------------ 1.2/12.8 MB 8.5 MB/s eta 0:00:02
     ----- ---------------------------------- 1.8/12.8 MB 9.4 MB/s eta 0:00:02
     ------- -------------------------------- 2.3/12.8 MB 9.7 MB/s eta 0:00:02
     -------- ------------------------------- 2.8/12.8 MB 9.9 MB/s eta 0:00:02
     ---------- ----------------------------- 3.4/12.8 MB 10.3 MB/s eta 0:00:01
     ------------ --------------------------- 3.9/12.8 MB 10.4 MB/s eta 0:00:01
     ------------- -------------------------- 4.4/12.8 MB 10.5 MB/s eta 0:00:01
     --------------- ------------------------ 5.0/12.8 MB 10.9 MB/s eta 0:00:01
     -----

For this notebook we will be using the following packages:

In [5]:
#Special module written for this class
#This provides access to data and to helper functions from previous weeks
import lucem_illud #pip install git+https://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git

#All these packages need to be installed from pip
import requests #for http requests
import bs4 #called `beautifulsoup4`, an html parser
import pandas #gives us DataFrames
import docx #reading MS doc files, install as `python-docx`

#Stuff for pdfs
#Install as `pdfminer2`
import pdfminer.pdfinterp
import pdfminer.converter
import pdfminer.layout
import pdfminer.pdfpage

#These come with Python
import re #for regexs
import urllib.parse #For joining urls
import io #for making http requests look like files
import json #For Tumblr API responses
import os.path #For checking if files exist
import os #For making directories

We will also be working on the following files/urls

In [33]:
wikipedia_base_url = 'https://en.wikipedia.org'
wikipedia_content_analysis = 'https://en.wikipedia.org/wiki/Content_analysis'
content_analysis_save = 'wikipedia_content_analysis.html'
example_text_file = 'sometextfile.txt'
information_extraction_pdf = 'https://github.com/Computational-Content-Analysis-2018/Data-Files/raw/master/1-intro/Content%20Analysis%2018.pdf'
example_docx = 'https://github.com/Computational-Content-Analysis-2018/Data-Files/raw/master/1-intro/macs6000_connecting_to_midway.docx'
example_docx_save = 'example.docx'

# Scraping

Before we can start analyzing content we need to obtain it. Sometimes it will be
provided to us from a pre-curated text archive, but sometimes we will need to
download it. As a starting example we will attempt to download the wikipedia
page on content analysis. The page is located at [https://en.wikipedia.org/wiki/
Content_analysis](https://en.wikipedia.org/wiki/Content_analysis) so lets start
with that.

We can do this by making an HTTP GET request to that url, a GET request is
simply a request to the server to provide the contents given by some url. The
other request we will be using in this class is called a POST request and
requests the server to take some content we provide. While the Python standard
library does have the ability do make GET requests we will be using the
[_requests_](http://docs.python-requests.org/en/master/) package as it is _'the
only Non-GMO HTTP library for Python'_...also it provides a nicer interface.

In [35]:
#wikipedia_content_analysis = 'https://en.wikipedia.org/wiki/Content_analysis'
requests.get(wikipedia_content_analysis)

<Response [200]>

`'Response [200]'` means the server responded with what we asked for. If you get
another number (e.g. 404) it likely means there was some kind of error, these
codes are called HTTP response codes and a list of them can be found
[here](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes). The response
object contains all the data the server sent including the website's contents
and the HTTP header. We are interested in the contents which we can access with
the `.text` attribute.

In [36]:
wikiContentRequest = requests.get(wikipedia_content_analysis)
print(wikiContentRequest.text[:1000])

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-zebra-design-enabled vector-feature-custom-font-size-clientpref-0 vector-feature-client-preferences-disabled vector-feature-client-prefs-pinned-disabled vector-toc-available" lang="en" dir="ltr">
<head>
<meta charset="UTF-8">
<title>Content analysis - Wikipedia</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-cli

This is not what we were looking for, because it is the start of the HTML that
makes up the website. This is HTML and is meant to be read by computers. Luckily
we have a computer to parse it for us. To do the parsing we will use [_Beautiful
Soup_](https://www.crummy.com/software/BeautifulSoup/) which is a better parser
than the one in the standard library.

But before we proceed to Beautiful Soup, a digression about Python syntax, especially about objects and functions.
For those who are not familiar with the syntax of python (or, if you're familiar with R programming), you might wonder what requests.get or wikiContentRequest.text mean. To understand this, you need to first understand what objects are. You may have heard that Python is an object oriented programming language (unlike the procedure oriented programming language, an example of which is R). Object is a set of variables (or, data) and functions into which you pass your data. So, in object oriented programming languages, like python, variables and functions are bunleded into objects.

For example, let's look at wikiContentRequest. We use dir() function, which returns the list of attributes and functions of objects.

In [ ]:
 dir(wikiContentRequest)

There's 'text' here. We used 'wikiContentRequest.text' to access 'text.' In other words, we use .(dot notation) to access functions from objects. wikiContentRequest has a set of functions, as shown above, and we used 'wikiContentRequest.text' to access one of them. By the way, dot notations do not necessarily refer to functions--it refers to anything that the entity contains. 



Moving on to the next step: BeautifulSoup, a Python library which extracts data from HTML and XML, and transforms HTML files into Python objects.

In [ ]:
wikiContentSoup = bs4.BeautifulSoup(wikiContentRequest.text, 'html.parser')
print(wikiContentSoup.text[:200])

This is better but there's still random whitespace and we have more than just
the text of the article. This is because what we requested is the whole webpage,
not just the text for the article.

We want to extract only the text we care about, and in order to do this we will
need to inspect the html. One way to do this is simply to go to the website with
a browser and use its inspection or view source tool. If javascript or other
dynamic loading occurs on the page, however, it is likely that what Python
receives is not what you will see, so we will need to inspect what Python
receives. To do this we can save the html `requests` obtained.

In [13]:
#content_analysis_save = 'wikipedia_content_analysis.html'

with open(content_analysis_save, mode='w', encoding='utf-8') as f:
    f.write(wikiContentRequest.text)

open() is a function which literally opens and returns the file. This function has multiple modes, and, here, we used mode as 'w', which means: open a file for writing. And then, we use 'write' function to write on the empty file (content_analysis_save) that we created using open(content_analysis_save, mode='w', encoding='utf-8').} What did we write on this file? The text we got from wikiContentRequest.text

Now let's open the file (`wikipedia_content_analysis.html`) we just created with
a web browser. It should look sort of like the original but without the images
and formatting.

As there is very little standardization on structuring webpages, figuring out
how best to extract what you want is an art. Looking at this page it looks like
all the main textual content is inside `<p>`(paragraph) tags within the `<body>`
tag. 

In [ ]:
contentPTags = wikiContentSoup.body.findAll('p')
for pTag in contentPTags[:3]:
    print(pTag.text)

Another excursion for those who are not familiar with programming: for loop. For loop is used to iterate over a sequence. "ContentPTags" contains multiple paragraphs, each of which starts and ends with `<p>`. What the "for pTag in contentPtags[:3]" does here is: find each paragraph in contentPTags, which, here, we limited to the first three using contentPtags[:3], and then print each paragraph. So, we have three paragraphs. By the way, you can insert `<p>` in juputer notebook!

We now have all the text from the page, split up by paragraph. If we wanted to
get the section headers or references as well it would require a bit more work,
but is doable.

There is one more thing we might want to do before sending this text to be
processed, remove the references indicators (`[2]`, `[3]` , etc). To do this we
can use a short regular expression (regex).

In [ ]:
contentParagraphs = []
for pTag in contentPTags:
    #strings starting with r are raw so their \'s are not modifier characters
    #If we didn't start with r the string would be: '\\[\\d+\\]'
    contentParagraphs.append(re.sub(r'\[\d+\]', '', pTag.text))

#convert to a DataFrame
contentParagraphsDF = pandas.DataFrame({'paragraph-text' : contentParagraphs})
print(contentParagraphsDF)

Since we learned how to do for loop, you might get what we did here: using contentParagraphs = [], we made an empty list; and then, for each paragraph in contentPTags, we substituted every [\d+\] with '', i.e., removed every [\d+\], and then appended each paragraph (now without [\d+\]) to the empty list. As we can see, we have a dataframe, each row of which is each paragraph of contentPTags, without reference indicators. 

By the way, what does [\d+\] mean? If you are not familiar with regex, it is a way of specifying searches in text.
A regex engine takes in the search pattern, in the above case `'\[\d+\]'` and
some string, the paragraph texts. Then it reads the input string one character
at a time checking if it matches the search. Here the regex `'\d'` matches
number characters (while `'\['` and `'\]'` capture the braces on either side).

Now we have a `DataFrame` containing all relevant text from the page ready to be processed

In [16]:
findNumber = r'\d'
regexResults = re.search(findNumber, 'not a number, not a number, numbers 2134567890, not a number')
regexResults

<re.Match object; span=(36, 37), match='2'>

In Python the regex package (`re`) usually returns `Match` objects (you can have
multiple pattern hits in a a single `Match`), to get the string that matched our
pattern we can use the `.group()` method, and as we want the first one we will
ask for the 0'th group.

In [17]:
print(regexResults.group(0))

2


That gives us the first number, if we wanted the whole block of numbers we can
add a wildcard `'+'` which requests 1 or more instances of the preceding
character.

In [18]:
findNumbers = r'\d+'
regexResults = re.search(findNumbers, 'not a number, not a number, numbers 2134567890, not a number')
print(regexResults.group(0))

2134567890


Now we have the whole block of numbers, there are a huge number of special
characters in regex, for the full description of Python's implementation look at
the [re docs](https://docs.python.org/3/library/re.html) there is also a short
[tutorial](https://docs.python.org/3/howto/regex.html#regex-howto).

# <font color="red">Exercise 1</font>
<font color="red">Construct cells immediately below this that describe and download webcontent relating to your anticipated final project. Use beautiful soup and at least five regular expressions to extract relevant, nontrivial *chunks* of that content (e.g., cleaned sentences, paragraphs, etc.) to a pandas `Dataframe`.</font>

##### A Sociological Dive into Classical Music Culture

 My curiosity was piqued by two fascinating books: 
 - *Mozart in the Jungle: Sex, Drugs, and Classical Music* by Blair Tindall and 
 - *Who Killed Classical Music?: Maestros, Managers, and Corporate Politics* by Norman Lebrecht. 
 
 In short, both are **non-fiction** which peel back the layers of glamour of music industry to reveal the raw, unpolished (dark) truth of the classical music realm.

In Exercise 1, I'll try to curate lists of books information (on *Goodreads*) that are related to these two books. I'll approach in two ways:
1. Scrape books that appear in the recommanded list of the original book
2. Directly scrape for all books on *Goodreads* that has tagged as *music* and *non-fiction*



https://www.goodreads.com/book/similar/25759-mozart-in-the-jungle-sex-drugs-and-classical-music?ref=rae_seeall

In [18]:
mozart_book_url = 'https://www.goodreads.com/book/show/24998.Mozart_in_the_Jungle' # Mozart in the Jungle book
killed_book_url = 'https://www.goodreads.com/book/show/792584.Who_Killed_Classical_Music' # Who Killed Classical Music? book
Bernstein_book_url = 'https://www.goodreads.com/book/show/36300698-famous-father-girl' # a Memoir of Leonard Bernstein's --- This is an audio book with no page numbers

In [23]:
def scrape_goodreads_book_page(url):
    '''
    This function works on the Main page of a Goodreads book.
    Now, it extracts the title, description, genres, rating, number of pages, format, and publication date.

    Input: url of the book page (string)
    Return: a dictionary of the book information
    
    '''
    # Send a request to the Goodreads book page
    response = requests.get(url)
    soup = bs4.BeautifulSoup(response.content, 'html.parser')

    # Extract the book title
    title = soup.find('h1', {'data-testid': 'bookTitle'}).get_text(strip=True)

    # Extract the book description
    description_div = soup.find('div', {'data-testid': 'description'})
    description = description_div.find('span', class_='Formatted').get_text(strip=True)

    # Extract the genres
    genres_div = soup.find('div', {'data-testid': 'genresList'})
    genres = [genre.get_text(strip=True) for genre in genres_div.find_all('span', class_='Button__labelItem')]

    # Extract the rating, ratings count, and reviews count
    rating = soup.find('div', class_='RatingStatistics__rating').get_text(strip=True)

    ratings_div = soup.find('div', class_='RatingStatistics__meta', role='figure')
    ratings_text = ratings_div['aria-label'] if ratings_div else ''

    ratings_count_match = re.search(r'([\d,]+) ratings', ratings_text)
    # when the number exceed 999, it displays as 1,000, so we need to replace the comma
    ratings_count = ratings_count_match.group(1).replace(',', '') if ratings_count_match else 'N/A'

    reviews_count_match = re.search(r'([\d,]+) reviews', ratings_text)
    reviews_count = reviews_count_match.group(1).replace(',', '') if reviews_count_match else 'N/A'

    # Extract the number of pages and format
    ## note: the page number and format are in the same <p> tag, and can be missing, use regex to extract them
    ## Things under that can be:
    ## -336 pages, Paperback
    ## -Audiobook
    details_div = soup.find('div', class_='FeaturedDetails')
    pages_format_text = details_div.find('p', {'data-testid': 'pagesFormat'}).get_text(strip=True)
    
    pages_match = re.search(r'(\d+) pages', pages_format_text) # look for numbers
    pages = pages_match.group(1) if pages_match else 'N/A'
    format_match = re.search(r'([A-Z][a-zA-Z]+)$', pages_format_text) # look for capital letter
    format_ = format_match.group(1) if format_match else 'N/A'

    # Extract the publication date using regex
    publication_info_text = details_div.find('p', {'data-testid': 'publicationInfo'}).get_text(strip=True)
    publication_date_match = re.search(r'published (.+)', publication_info_text) # need fix later (sometimes no "published" or capital letter)
    publication_date = publication_date_match.group(1) if publication_date_match else 'N/A'

    return {
        'title': title,
        'description': description,
        'genres': genres,
        'rating': rating,
        'ratings_count': ratings_count,
        'reviews_count': reviews_count,
        'pages': pages,
        'format': format_,
        'publication_date': publication_date
    }

In [24]:
# try the function on the Mozart book
mozart_info = scrape_goodreads_book_page(mozart_book_url)
mozart_info

{'title': 'Mozart in the Jungle: Sex, Drugs, and Classical Music',
 'description': 'In the tradition of Anthony Bourdain’s Kitchen Confidential and Gelsey Kirkland’s Dancing on My Grave , Mozart in the Jungle delves into the lives of the musicians and conductors who inhabit the insular world of classical music. In a book that inspired the Amazon Original series starring Gael García Bernal and Malcolm McDowell, oboist Blair Tindall recounts her decades-long professional career as a classical musician—from the recitals and Broadway orchestra performances to the secret life of musicians who survive hand to mouth in the backbiting New York classical music scene, where musicians trade sexual favors for plum jobs and assignments in orchestras across the city. Tindall and her fellow journeymen musicians often play drunk, high, or hopelessly hungover, live in decrepit apartments, and perform in hazardous conditions— working-class musicians who schlep across the city between low-paying gigs, wi

In [25]:
# try the function on the Bernstein book
Bernstain_info = scrape_goodreads_book_page(Bernstein_book_url)
Bernstain_info

{'title': 'Famous Father Girl: A Memoir of Growing Up Bernstein',
 'description': 'The oldest daughter of revered composer/conductor Leonard Bernstein offers a rare look at her father on the centennial of his birth in a deeply intimate and broadly evocative memoir.The composer ofOn the TownandWest Side Story, chief conductor of the New York Philharmonic, television star, humanitarian, friend of the powerful and influential, and the life of every party, Leonard Bernstein was an enormous celebrity during one of the headiest periods of American cultural life, as well as the most protean musician in twentieth century America.But to his eldest daughter, Jamie, he was above all the man in the scratchy brown bathrobe who smelled of cigarettes; the jokester and compulsive teacher who enthused about Beethoven and the Beatles; the insomniac whose 4 a.m. composing breaks involved spooning baby food out of the jar. He taught his daughter to love the world in all its beauty and complexity. In publi

In [26]:
def scrape_related_books(url):
    '''
    This scrapes a Goodreads page for a book's related books (readers also enjoyed).
    It can handle most pages that have a list of books with ratings and *descriptions*.

    NOTE: it cannot collect the full description; probably need selenium for expanding that.

    input: url (string) - the url of the Goodreads page
    returns: DataFrame - a DataFrame with the book's title, author, rating, number of ratings, and description

    '''
    response = requests.get(url)
    soup = bs4.BeautifulSoup(response.text, 'html.parser')
    
    books_data = []
    
    for book_div in soup.find_all('div', class_='responsiveBook'):
        # Extract title and link
        title_tag = book_div.find('a', class_='gr-h3 gr-h3--serif gr-h3--noMargin')
        title = title_tag.find('span', itemprop='name').get_text()
        link = 'https://www.goodreads.com' + title_tag['href']
        
        # Extract rating -- need regex to extract the number
        ## sample: <div class="communityRating" aria-label="Rated 3.96 of 5" .... > 
        ## need the number at position of 3.96
        rating_div = book_div.find('div', class_='communityRating')
        rating = re.search(r'Rated (\d+\.\d+) of 5', rating_div['aria-label']).group(1) 
        
        # Extract description
        description_span = book_div.find('div', class_='expandableHtml').find('span')
        description = description_span.get_text(strip=True)
        
        books_data.append({
            'Title': title,
            'Link': link,
            'Rating': rating,
            'Description': description
        })
    
    return pandas.DataFrame(books_data)

In [27]:
# Try the function on the Mozart book page
mozart_related_url = 'https://www.goodreads.com/book/similar/25759-mozart-in-the-jungle-sex-drugs-and-classical-music?ref=rae_seeall'

In [28]:
mozart_df = scrape_related_books(mozart_related_url)
mozart_df.head()

,Title,Link,Rating,Description
0,"Mozart in the Jungle: Sex, Drugs, and Classica...",https://www.goodreads.com/book/show/24998.Moza...,3.28,In the tradition of Anthony Bourdain’s Kitchen...
1,Famous Father Girl: A Memoir of Growing Up Ber...,https://www.goodreads.com/book/show/36300698-f...,3.80,The oldest daughter of revered composer/conduc...
2,The Ski Jumpers,https://www.goodreads.com/book/show/60240730-t...,3.96,A writer and former ski jumper facing a termin...
3,Swan Dive: The Making of a Rogue Ballerina,https://www.goodreads.com/book/show/53137979-s...,3.74,Award-winning New York City Ballet soloist Geo...
4,Two Women Walk into a Bar,https://www.goodreads.com/book/show/198712875-...,4.19,"Cheryl Strayed, the bestselling author of Wild..."


In [31]:
# Let try scraping all the Links of related books of the Mozart book and created a pandas DataFrame

links = mozart_df['Link'].tolist()

book_data_list = []

for url in links:
    try:
        book_data = scrape_goodreads_book_page(url)
        book_data_list.append(book_data)
    except Exception as e:
        print(f"Error scraping {url}: {e}")

# Convert the list of dictionaries to a DataFrame
books_df = pandas.DataFrame(book_data_list)


books_df
    


,title,description,genres,rating,ratings_count,reviews_count,pages,format,publication_date
0,"Mozart in the Jungle: Sex, Drugs, and Classica...",In the tradition of Anthony Bourdain’s Kitchen...,"[Music, Nonfiction, Memoir, Biography, Biograp...",3.38,2963,404,336,Paperback,"January 1, 2005"
1,Famous Father Girl: A Memoir of Growing Up Ber...,The oldest daughter of revered composer/conduc...,"[Memoir, Nonfiction, Biography, Music, Biograp...",3.88,1271,197,N/A,Audiobook,"January 1, 2018"
2,The Ski Jumpers,A writer and former ski jumper facing a termin...,"[Fiction, Contemporary]",3.99,280,46,408,Hardcover,"September 13, 2022"
3,Swan Dive: The Making of a Rogue Ballerina,Award-winning New York City Ballet soloist Geo...,"[Nonfiction, Memoir, Biography, Audiobook, Spo...",3.82,1744,261,272,Hardcover,"July 27, 2021"
4,Two Women Walk into a Bar,"Cheryl Strayed, the bestselling author of Wild...","[Nonfiction, Memoir, Short Stories, Biography,...",4.19,4763,376,31,Edition,N/A
5,The Piano Shop on the Left Bank: Discovering a...,Walking his two young children to school every...,"[Nonfiction, Music, Memoir, France, Travel, Bi...",3.97,6196,821,304,Edition,"January 1, 2000"
6,Rising: Dispatches from the New American Shore,WINNER OF THE NATIONAL OUTDOOR BOOK AWARDACHIC...,"[Nonfiction, Science, Environment, Nature, Cli...",4.18,2218,390,299,Hardcover,"June 12, 2018"
7,The Tumbler,"Gabriel Du Pré, the old Métis fiddler at the c...","[Mystery, Westerns, Fiction, Mystery Thriller,...",4.25,251,13,256,Edition,"April 12, 2004"
8,The Well-Educated Mind: A Guide to the Classic...,Have you lost the art of reading for pleasure?...,"[Education, Nonfiction, Books About Books, Ref...",3.99,3593,413,432,Hardcover,"January 1, 2003"
9,Year of Wonder: Classical Music for Every Day,Have you ever heard a piece of music so beauti...,"[Music, Nonfiction, History, Reference, Art, E...",4.41,1093,214,416,Hardcover,"October 5, 2017"


##### Looks like it's not a good idea to start from the recommanded lists of a book
- This list changed when I scrape it the next day
- not all the books there are related to *music*

In [ ]:
# https://www.goodreads.com/list/show/544.Best_Non_Fiction_About_Music?page=1


# Spidering

What if we want to to get a bunch of different pages from wikipedia. We would
need to get the url for each of the pages we want. Typically, we want pages that
are linked to by other pages and so we will need to parse pages and identify the
links. Right now we will be retrieving all links in the body of the content
analysis page.

To do this we will need to find all the `<a>` (anchor) tags with `href`s
(hyperlink references) inside of `<p>` tags. `href` can have many
[different](http://stackoverflow.com/questions/4855168/what-is-href-and-why-is-
it-used) [forms](https://en.wikipedia.org/wiki/Hyperlink#Hyperlinks_in_HTML) so
dealing with them can be tricky, but generally, you will want to extract
absolute or relative links. An absolute link is one you can follow without
modification, while a relative link requires a base url that you will then
append. Wikipedia uses relative urls for its internal links: below is an example
for dealing with them.

In [41]:
#wikipedia_base_url = 'https://en.wikipedia.org'

otherPAgeURLS = []
#We also want to know where the links come from so we also will get:
#the paragraph number
#the word the link is in
for paragraphNum, pTag in enumerate(contentPTags):
    #we only want hrefs that link to wiki pages
    tagLinks = pTag.findAll('a', href=re.compile('/wiki/'), class_=False)
    for aTag in tagLinks:
        #We need to extract the url from the <a> tag
        relurl = aTag.get('href')
        linkText = aTag.text
        #wikipedia_base_url is the base we can use the urllib joining function to merge them
        #Giving a nice structured tupe like this means we can use tuple expansion later
        otherPAgeURLS.append((
            urllib.parse.urljoin(wikipedia_base_url, relurl),
            paragraphNum,
            linkText,
        ))
print(otherPAgeURLS[:10])

[('https://en.wikipedia.org/wiki/Document', 0, 'documents'), ('https://en.wikipedia.org/wiki/Text_(literary_theory)', 1, 'texts'), ('https://en.wikipedia.org/wiki/Coding_(social_sciences)', 1, 'assigned labels (sometimes called codes)'), ('https://en.wikipedia.org/wiki/Semantics', 1, 'meaningful'), ('https://en.wikipedia.org/wiki/Text_(literary_theory)', 1, 'texts'), ('https://en.wikipedia.org/wiki/Quantitative_research', 1, 'quantitatively'), ('https://en.wikipedia.org/wiki/Statistics', 1, 'statistical methods'), ('https://en.wikipedia.org/wiki/Qualitative_research', 1, 'qualitative'), ('https://en.wikipedia.org/wiki/Text_(literary_theory)', 1, 'texts'), ('https://en.wikipedia.org/wiki/Machine_learning', 2, 'Machine learning')]


In [ ]:
print(contentPTags)

Another excursion: Why do we use enumerate() here? enumerate() takes a collection, enumerates, and returns an enumate object with both the numbers and the collection. For example, contentPTags (the collection we used here) is comprised of paragraphs. We want the paragraph number of each paragraph. And this is what enumerate() does: it returns the paragraph number and the paragraph. 

We will be adding these new texts to our DataFrame `contentParagraphsDF` so we
will need to add 2 more columns to keep track of paragraph numbers and sources.

In [43]:
contentParagraphsDF['source'] = [wikipedia_content_analysis] * len(contentParagraphsDF['paragraph-text'])
contentParagraphsDF['paragraph-number'] = range(len(contentParagraphsDF['paragraph-text']))

contentParagraphsDF

,paragraph-text,source,paragraph-number
0,Content analysis is the study of documents and...,https://en.wikipedia.org/wiki/Content_analysis,0
1,Practices and philosophies of content analysis...,https://en.wikipedia.org/wiki/Content_analysis,1
2,Computers are increasingly used in content ana...,https://en.wikipedia.org/wiki/Content_analysis,2
3,Content analysis is best understood as a broad...,https://en.wikipedia.org/wiki/Content_analysis,3
4,The simplest and most objective form of conten...,https://en.wikipedia.org/wiki/Content_analysis,4
5,A further step in analysis is the distinction ...,https://en.wikipedia.org/wiki/Content_analysis,5
6,Quantitative content analysis highlights frequ...,https://en.wikipedia.org/wiki/Content_analysis,6
7,Siegfried Kracauer provides a critique of quan...,https://en.wikipedia.org/wiki/Content_analysis,7
8,The data collection instrument used in content...,https://en.wikipedia.org/wiki/Content_analysis,8
9,According to current standards of good scienti...,https://en.wikipedia.org/wiki/Content_analysis,9


Then we can add two more columns to our `Dataframe` and define a function to
parse
each linked page and add its text to our DataFrame.

In [44]:
contentParagraphsDF['source-paragraph-number'] = [None] * len(contentParagraphsDF['paragraph-text'])
contentParagraphsDF['source-paragraph-text'] = [None] * len(contentParagraphsDF['paragraph-text'])

def getTextFromWikiPage(targetURL, sourceParNum, sourceText):
    #Make a dict to store data before adding it to the DataFrame
    parsDict = {'source' : [], 'paragraph-number' : [], 'paragraph-text' : [], 'source-paragraph-number' : [],  'source-paragraph-text' : []}
    #Now we get the page
    r = requests.get(targetURL)
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    #enumerating gives use the paragraph number
    for parNum, pTag in enumerate(soup.body.findAll('p')):
        #same regex as before
        parsDict['paragraph-text'].append(re.sub(r'\[\d+\]', '', pTag.text))
        parsDict['paragraph-number'].append(parNum)
        parsDict['source'].append(targetURL)
        parsDict['source-paragraph-number'].append(sourceParNum)
        parsDict['source-paragraph-text'].append(sourceText)
    return pandas.DataFrame(parsDict)

And run it on our list of link tags

In [ ]:
for urlTuple in otherPAgeURLS[:3]:
    #ignore_index means the indices will not be reset after each append
    contentParagraphsDF = contentParagraphsDF.append(getTextFromWikiPage(*urlTuple),ignore_index=True)
contentParagraphsDF


# <font color="red">Exercise 2</font>
<font color="red">Construct cells immediately below this that spider webcontent from another site with content relating to your anticipated final project. Specifically, identify urls on a core page, then follow and extract content from them into a pandas `Dataframe`. In addition, demonstrate a *recursive* spider, which follows more than one level of links (i.e., follows links from a site, then follows links on followed sites to new sites, etc.), making sure to define a reasonable endpoint so that you do not wander the web forever :-).</font>



In [66]:
def is_goodreads_book_link(url):
    """
    Checks if the URL is a valid Goodreads book link.
    """
    parsed_url = urllib.parse.urlparse(url)
    return parsed_url.netloc == 'www.goodreads.com' and '/book/show/' in parsed_url.path

def scrape_book_info(url):
    """
    Scrapes the book information from a Goodreads book page.
    """
    response = requests.get(url)
    soup = bs4.BeautifulSoup(response.content, 'html.parser')
    
    # use same code in exercise 1 to extract book genre
    # we only return when it is a music book
    try:
        genres_div = soup.find('div', {'data-testid': 'genresList'})
        genres = [genre.get_text(strip=True) for genre in genres_div.find_all('span', class_='Button__labelItem')]
        for genre in genres:
            if 'music' in genre.lower(): # any tag containing 'music' counts (e.g. 'classical music', "music-history")
                return {
                    'url': url,
                    'title': soup.find('h1', {'data-testid': 'bookTitle'}).get_text(strip=True),
                    # Add other book details as needed
                }
    except:
        return None


    return None


In [59]:
a = scrape_book_info(mozart_book_url)
a

{'url': 'https://www.goodreads.com/book/show/24998.Mozart_in_the_Jungle',
 'title': 'Mozart in the Jungle: Sex, Drugs, and Classical Music'}

In [53]:
type(a)

NoneType

In [70]:
def crawl_goodreads(start_url, max_depth=5):
    """
    Crawls Goodreads starting from a given book page URL.
    """
    visited = set()
    books_to_visit = [(start_url, 0)]
    collected_books = []

    while books_to_visit:
        current_url, current_depth = books_to_visit.pop(0)
        if current_url not in visited and current_depth <= max_depth:
            visited.add(current_url)
            response = requests.get(current_url)
            soup = bs4.BeautifulSoup(response.content, 'html.parser')

            book_info = scrape_book_info(current_url)
            if book_info:
                collected_books.append(book_info)

            if current_depth < max_depth:
                # <div class="BookCard">
                recommended_books = soup.find_all('div', class_='BookCard')
                for link in recommended_books:
                    href = link['href']
                    full_url = urllib.parse.urljoin(current_url, href)
                    if is_goodreads_book_link(full_url):
                        books_to_visit.append((full_url, current_depth + 1))

    return collected_books


In [75]:
response = requests.get(mozart_book_url)
soup = bs4.BeautifulSoup(response.content, 'html.parser')
#<div class="OverflowButton__desktop"

urls = []
for link in soup.find_all('a', href=True):
    href = link['href']
    urls.append(href)

urls

# recommended_books = soup.find_all('div', class_="OverfllowButton__desktop")
# recommended_books

['https://www.goodreads.com/challenges/11634?ref=rc_jan_24_ss',
 '/?ref=nav_hom',
 '/?ref=nav_hom',
 '/review/list?ref=nav_mybooks',
 '#?ref=nav_brws',
 '/recommendations?ref=nav_brws_recs',
 '/choiceawards?ref=nav_brws_gca',
 '/giveaway?ref=nav_brws_giveaways',
 '/new_releases?ref=nav_brws_newrels',
 '/list?ref=nav_brws_lists',
 '/book?ref=nav_brws_explore',
 '/news?ref=nav_brws_news',
 '#?ref=nav_comm',
 '/group?ref=nav_comm_groups',
 '/quotes?ref=nav_comm_quotes',
 '/ask_the_author?ref=nav_comm_askauthor',
 '/user/best_reviewers?ref=nav_comm_people',
 '/user/sign_in',
 '/user/sign_up',
 '#ReviewsSection',
 'https://www.goodreads.com/author/show/14044.Blair_Tindall',
 '#CommunityReviews',
 'https://www.goodreads.com/genres/music',
 'https://www.goodreads.com/genres/non-fiction',
 'https://www.goodreads.com/genres/memoir',
 'https://www.goodreads.com/genres/biography',
 'https://www.goodreads.com/genres/biography-memoir',
 'https://www.goodreads.com/genres/autobiography',
 'https://ww

In [71]:
books = crawl_goodreads(mozart_book_url, max_depth=2)

KeyError: 'href'

In [78]:
# Every book in Goodreads has a unique ID, but multiple URLs can point to the same book.
# record id so recursive spidering doesn't visit the same book
def extract_book_id(url):
    match = re.search(r'/book/show/(\d+)', url)
    return match.group(1) if match else None

In [79]:
def is_goodreads_book_link(url):
    return 'https://www.goodreads.com/book/show/' in url

def is_goodreads_author_link(url):
    return 'https://www.goodreads.com/author/show/' in url

In [106]:
def get_genres(soup):
    '''
    Same but shortened from exercise 1
    return a list of genres, so we can check if it is a music book
    '''
    genres_div = soup.find('div', {'data-testid': 'genresList'})
    if genres_div:
        return [genre.get_text(strip=True).lower() for genre in genres_div.find_all('span', class_='Button__labelItem')]
    return []

def scrape_book_info(url):
    response = requests.get(url)
    soup = bs4.BeautifulSoup(response.content, 'html.parser')
    genres = get_genres(soup)

    if 'music' in genres:
        title = soup.find('h1', {'data-testid': 'bookTitle'}).get_text(strip=True)
        author_link = soup.find('a', href=lambda href: href and is_goodreads_author_link(href))
        return {'title': title, 'url': url, 'author_link': author_link['href'] if author_link else None}
    return None

def crawl_author_books(author_url, visited_books):
    response = requests.get(author_url)
    soup = bs4.BeautifulSoup(response.content, 'html.parser')
    book_links = soup.find_all('a', href=lambda href: href and is_goodreads_book_link(href))

    print(f'Found {len(book_links)} books for {author_url}')

    author_books = []
    for link in book_links:
        book_url = urllib.parse.urljoin(author_url, link['href'])
        book_id = extract_book_id(book_url)

        if book_id and (book_id not in visited_books):
            visited_books.add(book_id)
            book_info = scrape_book_info(book_url)
            if book_info:
                author_books.append(book_info)
    return author_books


In [103]:
def crawl_goodreads(start_url):
    visited_books = set()
    # read from local html file, start_url is the file path
    content = open(start_url, 'r').read()
    soup = bs4.BeautifulSoup(content, 'html.parser')  

    aTags = soup.find_all('a', href=lambda href: href and is_goodreads_book_link(href))
    queue = []
    for tag in aTags:
        link = urllib.parse.urljoin(url, tag['href'])
        queue.append(link)
        # let's only crawl 10 books
        if len(queue) > 10:
            break

    collected_books = []

    while queue:
        current_url = queue.pop(0)
        book_id = extract_book_id(current_url)
        if book_id and (book_id not in visited_books):
            visited_books.add(book_id)
            book_info = scrape_book_info(current_url)
            if book_info:
                collected_books.append(book_info)
                if book_info['author_link']:
                    author_books = crawl_author_books(book_info['author_link'], visited_books)
                    collected_books.extend(author_books)
                    queue.extend(book['url'] for book in author_books if book['url'] not in visited_books)

    return collected_books

In [83]:
non_fiction_music_url = 'https://www.goodreads.com/list/show/544.Best_Non_Fiction_About_Music?page=1'
url ='https://www.goodreads.com/search?q=music&qid='

In [ ]:
downloaded_html = 'www.goodreads.com_list_show_544.Best_Non_Fiction_About_Music_page=1.html'
content = open(downloaded_html, 'r').read()
soup = bs4.BeautifulSoup(content, 'html.parser')  
aTags = soup.find_all('a', href=lambda href: href and is_goodreads_book_link(href))

urls = []
for tag in aTags:
    urls.append(urllib.parse.urljoin(url, tag['href']))

urls

In [92]:
#extract all hrefs from links
urls = []
for link in links:
    href = link['href']
    urls.append(href)
urls

['https://www.goodreads.com/book/show/6587454.Big_Man_Real_Life___Tall_Tales',
 'https://www.goodreads.com/book/show/1388694.High_Fidelity',
 'https://www.goodreads.com/book/show/25378.The_Dirt_Confessions_of_the_World_s_Most_Notorious_Rock_Band',
 'https://www.goodreads.com/book/show/13623901.Kicking_and_Dreaming_A_Story_of_Heart__Soul__and_Rock_and_Roll',
 'https://www.goodreads.com/book/show/86702.C_mon__Get_Happy_Fear_and_Loathing_on_the_Partridge_Family_Bus',
 'https://www.goodreads.com/book/show/29072594.Born_to_Run',
 'https://www.goodreads.com/book/show/44303730.Me',
 'https://www.goodreads.com/book/show/279845.Conversations_with_Tom_Petty',
 'https://www.goodreads.com/book/show/12953257.A_Natural_Woman_A_Memoir',
 'https://www.goodreads.com/book/show/28686968.Not_Dead_Yet_The_Memoir',
 'https://www.goodreads.com/book/show/44418422.Wham__George___Me',
 'https://www.goodreads.com/book/show/25361912.Over_the_Top_and_Back_The_Autobiography',
 'https://www.goodreads.com/book/show/2

In [108]:
books = crawl_goodreads(downloaded_html)
books

Found 0 books for https://www.goodreads.com/author/show/9097.Legs_McNeil
Found 6 books for https://www.goodreads.com/author/show/196092.Patti_Smith
Found 0 books for https://www.goodreads.com/author/show/8898.Bob_Dylan
Found 10 books for https://www.goodreads.com/author/show/45539.Keith_Richards
Found 0 books for https://www.goodreads.com/author/show/16495.Michael_Azerrad
Found 0 books for https://www.goodreads.com/author/show/25839.Rob_Sheffield


[{'title': 'Please Kill Me: The Uncensored Oral History of Punk',
  'url': 'https://www.goodreads.com/book/show/14595.Please_Kill_Me',
  'author_link': 'https://www.goodreads.com/author/show/9097.Legs_McNeil'},
 {'title': 'Just Kids',
  'url': 'https://www.goodreads.com/book/show/341879.Just_Kids',
  'author_link': 'https://www.goodreads.com/author/show/196092.Patti_Smith'},
 {'title': 'Chronicles, Volume One',
  'url': 'https://www.goodreads.com/book/show/14318.Chronicles_Volume_One',
  'author_link': 'https://www.goodreads.com/author/show/8898.Bob_Dylan'},
 {'title': 'Life',
  'url': 'https://www.goodreads.com/book/show/9439303-life',
  'author_link': 'https://www.goodreads.com/author/show/45539.Keith_Richards'},
 {'title': 'Our Band Could Be Your Life: Scenes from the American Indie Underground 1981-1991',
  'url': 'https://www.goodreads.com/book/show/29393.Our_Band_Could_Be_Your_Life',
  'author_link': 'https://www.goodreads.com/author/show/16495.Michael_Azerrad'},
 {'title': 'Love

Look like I need to use selelium 

## API (Tumblr)

Generally website owners do not like you scraping their sites. If done badly,
scarping can act like a DOS attack so you should be careful how often you make
calls to a site. Some sites want automated tools to access their data, so they
create [application programming interface
(APIs)](https://en.wikipedia.org/wiki/Application_programming_interface). An API
specifies a procedure for an application (or script) to access their data. Often
this is though a [representational state transfer
(REST)](https://en.wikipedia.org/wiki/Representational_state_transfer) web
service, which just means if you make correctly formatted HTTP requests they
will return nicely formatted data.

A nice example for us to study is [Tumblr](https://www.tumblr.com), they have a
[simple RESTful API](https://www.tumblr.com/docs/en/api/v1) that allows you to
read posts without any complicated html parsing.

We can get the first 20 posts from a blog by making an http GET request to
`'http://{blog}.tumblr.com/api/read/json'`, were `{blog}` is the name of the
target blog. Lets try and get the posts from [http://lolcats-lol-
cat.tumblr.com/](http://lolcats-lol-cat.tumblr.com/) (Note the blog says at the
top 'One hour one pic lolcats', but the canonical name that Tumblr uses is in
the URL 'lolcats-lol-cat').

In [ ]:
tumblrAPItarget = 'http://{}.tumblr.com/api/read/json'

r = requests.get(tumblrAPItarget.format('lolcats-lol-cat'))

print(r.text[:1000])

This might not look very good on first inspection, but it has far fewer angle
braces than html, which makes it easier to parse. What we have is
[JSON](https://en.wikipedia.org/wiki/JSON) a 'human readable' text based data
transmission format based on javascript. Luckily, we can readily convert it to a
python `dict`.

In [ ]:
#We need to load only the stuff between the curly braces
d = json.loads(r.text[len('var tumblr_api_read = '):-2])
print(d.keys())
print(len(d['posts']))

If we read the [API specification](https://www.tumblr.com/docs/en/api/v1), we
will see there are a lot of things we can get if we add things to our GET
request. First we can retrieve posts by their id number. Let's first get post
`146020177084`.

In [ ]:
r = requests.get(tumblrAPItarget.format('lolcats-lol-cat'), params = {'id' : 146020177084})
d = json.loads(r.text[len('var tumblr_api_read = '):-2])
d['posts'][0].keys()
d['posts'][0]['photo-url-1280']

with open('lolcat.gif', 'wb') as f:
    gifRequest = requests.get(d['posts'][0]['photo-url-1280'], stream = True)
    f.write(gifRequest.content)

<img src='lolcat.gif'>

Such beauty; such vigor (If you can't see it you have to refresh the page). Now
we could retrieve the text from all posts as well
as related metadata, like the post date, caption or tags. We could also get
links to all the images.

In [ ]:
#Putting a max in case the blog has millions of images
#The given max will be rounded up to the nearest multiple of 50
def tumblrImageScrape(blogName, maxImages = 200):
    #Restating this here so the function isn't dependent on any external variables
    tumblrAPItarget = 'http://{}.tumblr.com/api/read/json'

    #There are a bunch of possible locations for the photo url
    possiblePhotoSuffixes = [1280, 500, 400, 250, 100]

    #These are the pieces of information we will be gathering,
    #at the end we will convert this to a DataFrame.
    #There are a few other datums we could gather like the captions
    #you can read the Tumblr documentation to learn how to get them
    #https://www.tumblr.com/docs/en/api/v1
    postsData = {
        'id' : [],
        'photo-url' : [],
        'date' : [],
        'tags' : [],
        'photo-type' : []
    }

    #Tumblr limits us to a max of 50 posts per request
    for requestNum in range(maxImages // 50):
        requestParams = {
            'start' : requestNum * 50,
            'num' : 50,
            'type' : 'photo'
        }
        r = requests.get(tumblrAPItarget.format(blogName), params = requestParams)
        requestDict = json.loads(r.text[len('var tumblr_api_read = '):-2])
        for postDict in requestDict['posts']:
            #We are dealing with uncleaned data, we can't trust it.
            #Specifically, not all posts are guaranteed to have the fields we want
            try:
                postsData['id'].append(postDict['id'])
                postsData['date'].append(postDict['date'])
                postsData['tags'].append(postDict['tags'])
            except KeyError as e:
                raise KeyError("Post {} from {} is missing: {}".format(postDict['id'], blogName, e))

            foundSuffix = False
            for suffix in possiblePhotoSuffixes:
                try:
                    photoURL = postDict['photo-url-{}'.format(suffix)]
                    postsData['photo-url'].append(photoURL)
                    postsData['photo-type'].append(photoURL.split('.')[-1])
                    foundSuffix = True
                    break
                except KeyError:
                    pass
            if not foundSuffix:
                #Make sure your error messages are useful
                #You will be one of the users
                raise KeyError("Post {} from {} is missing a photo url".format(postDict['id'], blogName))

    return pandas.DataFrame(postsData)
tumblrImageScrape('lolcats-lol-cat', 50)

Now we have the urls of a bunch of images and can run OCR on them to gather
compelling meme narratives, accompanied by cats.

# Files

What if the text we want isn't on a webpage? There are a many other sources of
text available, typically organized into *files*.

## Raw text (and encoding)

The most basic form of storing text is as a _raw text_ document. Source code
(`.py`, `.r`, etc) is usually raw text as are text files (`.txt`) and those with
many other extension (e.g., .csv, .dat, etc.). Opening an unknown file with a
text editor is often a great way of learning what the file is.

We can create a text file in python with the `open()` function

In [ ]:
#example_text_file = 'sometextfile.txt'
#stringToWrite = 'A line\nAnother line\nA line with a few unusual symbols \u2421 \u241B \u20A0 \u20A1 \u20A2 \u20A3 \u0D60\n'
stringToWrite = 'A line\nAnother line\nA line with a few unusual symbols ␡ ␛ ₠ ₡ ₢ ₣ ൠ\n'

with open(example_text_file, mode = 'w', encoding='utf-8') as f:
    f.write(stringToWrite)

Notice the `encoding='utf-8'` argument, which specifies how we map the bits from
the file to the glyphs (and whitespace characters like tab (`'\t'`) or newline
(`'\n'`)) on the screen. When dealing only with latin letters, arabic numerals
and the other symbols on America keyboards you usually do not have to worry
about encodings as the ones used today are backwards compatible with
[ASCII](https://en.wikipedia.org/wiki/ASCII), which gives the binary
representation of 128 characters.

Some of you, however, will want to use other characters (e.g., Chinese
characters). To solve this there is
[Unicode](https://en.wikipedia.org/wiki/Unicode) which assigns numbers to
symbols, e.g., 041 is `'A'` and 03A3 is `'Σ'` (numbers starting with 0 are
hexadecimal). Often non/beyond-ASCII characters are called Unicode characters.
Unicode contains 1,114,112 characters, about 10\% of which have been assigned.
Unfortunately there are many ways used to map combinations of bits to Unicode
symbols. The ones you are likely to encounter are called by Python _utf-8_,
_utf-16_ and _latin-1_. _utf-8_ is the standard for Linux and Mac OS while both
_utf-16_ and _latin-1_ are used by windows. If you use the wrong encoding,
characters can appear wrong, sometimes change in number or Python could raise an
exception. Lets see what happens when we open the file we just created with
different encodings.

In [ ]:
with open(example_text_file, encoding='utf-8') as f:
    print("This is with the correct encoding:")
    print(f.read())

with open(example_text_file, encoding='latin-1') as f:
    print("This is with the wrong encoding:")
    print(f.read())

Notice that with _latin-1_ the unicode characters are mixed up and there are too
many of them. You need to keep in mind encoding when obtaining text files.
Determining the encoding can sometime involve substantial work.

We can also load many text files at once. Lets start by looking at the Shakespeare files in the `data` directory 

In [ ]:
with open('./data/Shakespeare/midsummer_nights_dream.txt') as f:
    midsummer = f.read()
print(midsummer[-700:])

By the way, depending on your working directory, you might get errors such as: [Errno 2] No such file or directory: '../data/Shakespeare/midsummer_nights_dream.txt.' Don't panic, it's nothing, just check your working directory. 

Then to load all the files in `./data/Shakespeare` we can use a for loop with `scandir`:

In [ ]:
targetDir = './data/Shakespeare' #Change this to your own directory of texts
shakespearText = []
shakespearFileName = []

for file in (file for file in os.scandir(targetDir) if file.is_file() and not file.name.startswith('.')):
    with open(file.path, encoding="utf-8") as f:
        shakespearText.append(f.read())
    shakespearFileName.append(file.name)

Then we can put them all in pandas DataFrame

In [ ]:
shakespear_df = pandas.DataFrame({'text' : shakespearText}, index = shakespearFileName)
shakespear_df

Getting your text in a format like this is the first step of most analysis

## PDF

Another common way text will be stored is in a PDF file. First we will download
a pdf in Python. To do that lets grab a chapter from
_Speech and Language Processing_, chapter 21 is on Information Extraction which
seems apt. It is stored as a pdf at [https://web.stanford.edu/~jurafsky/slp3/21.
pdf](https://web.stanford.edu/~jurafsky/slp3/21.pdf) although we are downloading
from a copy just in case Jurafsky changes their website.

In [ ]:
#information_extraction_pdf = 'https://github.com/KnowledgeLab/content_analysis/raw/data/21.pdf'

infoExtractionRequest = requests.get(information_extraction_pdf, stream=True)
print(infoExtractionRequest.text[:1000])

It says `'pdf'`, so thats a good sign. The rest though looks like we are having
issues with an encoding. The random characters are not caused by our encoding
being wrong, however. They are cause by there not being an encoding for those
parts at all. PDFs are nominally binary files, meaning there are sections of
binary that are specific to pdf and nothing else so you need something that
knows about pdf to read them. To do that we will be using
[`PyPDF2`](https://github.com/mstamy2/PyPDF2), a PDF processing library for
Python 3.


Because PDFs are a very complicated file format pdfminer requires a large amount
of boilerplate code to extract text, we have written a function that takes in an
open PDF file and returns the text so you don't have to.

In [ ]:
def readPDF(pdfFile):
    #Based on code from http://stackoverflow.com/a/20905381/4955164
    #Using utf-8, if there are a bunch of random symbols try changing this
    codec = 'utf-8'
    rsrcmgr = pdfminer.pdfinterp.PDFResourceManager()
    retstr = io.StringIO()
    layoutParams = pdfminer.layout.LAParams()
    device = pdfminer.converter.TextConverter(rsrcmgr, retstr, laparams = layoutParams, codec = codec)
    #We need a device and an interpreter
    interpreter = pdfminer.pdfinterp.PDFPageInterpreter(rsrcmgr, device)
    password = ''
    maxpages = 0
    caching = True
    pagenos=set()
    for page in pdfminer.pdfpage.PDFPage.get_pages(pdfFile, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)
    device.close()
    returnedString = retstr.getvalue()
    retstr.close()
    return returnedString

First we need to take the response object and convert it into a 'file like'
object so that pdfminer can read it. To do this we will use `io`'s `BytesIO`.

In [ ]:
infoExtractionBytes = io.BytesIO(infoExtractionRequest.content)

Now we can give it to pdfminer.

In [ ]:
print(readPDF(infoExtractionBytes)[:550])

From here we can either look at the full text or fiddle with our PDF reader and
get more information about individual blocks of text.

## Word Docs

The other type of document you are likely to encounter is the `.docx`, these are
actually a version of [XML](https://en.wikipedia.org/wiki/Office_Open_XML), just
like HTML, and like HTML we will use a specialized parser.

For this class we will use [`python-docx`](https://python-
docx.readthedocs.io/en/latest/) which provides a nice simple interface for
reading `.docx` files

In [ ]:
#example_docx = 'https://github.com/KnowledgeLab/content_analysis/raw/data/example_doc.docx'

r = requests.get(example_docx, stream=True)
d = docx.Document(io.BytesIO(r.content))
for paragraph in d.paragraphs[:7]:
    print(paragraph.text)

This procedure uses the `io.BytesIO` class again, since `docx.Document` expects
a file. Another way to do it is to save the document to a file and then read it
like any other file. If we do this we can either delete the file afterwords, or
save it and avoid downloading the following time.

This function is useful as a part of many different tasks so it and others like it will be added to the helper package `lucem_illud` so we can use it later without having to retype it.

In [ ]:
def downloadIfNeeded(targetURL, outputFile, **openkwargs):
    if not os.path.isfile(outputFile):
        outputDir = os.path.dirname(outputFile)
        #This function is a more general os.mkdir()
        if len(outputDir) > 0:
            os.makedirs(outputDir, exist_ok = True)
        r = requests.get(targetURL, stream=True)
        #Using a closure like this is generally better than having to
        #remember to close the file. There are ways to make this function
        #work as a closure too
        with open(outputFile, 'wb') as f:
            f.write(r.content)
    return open(outputFile, **openkwargs)

This function will download, save and open `outputFile` as `outputFile` or just
open it if `outputFile` exists. By default `open()` will open the file as read
only text with the local encoding, which may cause issues if its not a text
file.

In [ ]:
try:
    d = docx.Document(downloadIfNeeded(example_docx, example_docx_save))
except Exception as e:
    print(e)

We need to tell `open()` to read in binary mode (`'rb'`), this is why we added
`**openkwargs`, this allows us to pass any keyword arguments (kwargs) from
`downloadIfNeeded` to `open()`.

In [ ]:
d = docx.Document(downloadIfNeeded(example_docx, example_docx_save, mode = 'rb'))
for paragraph in d.paragraphs[:7]:
    print(paragraph.text)

Now we can read the file with `docx.Document` and not have to wait for it to be
downloaded every time.

# <font color="red">Exercise 3</font>
<font color="red">Construct cells immediately below this that extract and organize textual content from text, PDF or Word into a pandas dataframe.</font>


# <font color="red">Exercise 4</font>

<font color="red">In the two cells immediately following, describe a possible project (e.g., it might end up being your final project, but need not be if you are still searching): **WHAT** you will analyze--the texts you will select and the social game, world and actors you intend to learn about through your analysis (<100 words); **WHY** you will analyze these texts to learn about that context--justify the rationale behind your proposed sample design for this project, based on the readings. What is the social game, social work, or social actors about whom you are seeking to make inferences? What are the virtues of your proposed sample with respect to your research questions? What are its limitations? What are alternatives? What would be a reasonable path to "scale up" your sample for further analysis (i.e., high-profile publication)? (<150 words)? [**Note**: your individual or collective project will change over the course of the quarter as new data and/or analysis opportunities arise or if old ones fade away.] 


**Research Interest & Questions**
My research is rooted in exploring the sociological aspects of music, particularly classical. I'm captivated by how non-fiction literature critiques and demystifies the elite aura surrounding classical music. The question at the heart of my investigation is: *Is this theme of unveiling the darker side of classical music a common thread in related literature?*

In [76]:


**Research Interest & Questions**
My research is rooted in exploring the sociological aspects of music, particularly classical. I'm captivated by how non-fiction literature critiques and demystifies the elite aura surrounding classical music. The question at the heart of my investigation is: *Is this theme of unveiling the darker side of classical music a common thread in related literature?*

SyntaxError: unterminated string literal (detected at line 2) (4043517621.py, line 2)

## ***What?*** 
<100 words

## ***Why?***
<150 words

### Sources:

Other popular sources for internet data:

[reddit](https://www.reddit.com/) - https://praw.readthedocs.io/en/v2.1.21/

[twitter](https://twitter.com/) - https://pypi.org/project/python-twitter/

[project gutenburg](https://www.gutenberg.org/) - https://github.com/ageitgey/Gutenberg 

